<a href="https://colab.research.google.com/github/ccspen21/greenland-fishery-nowcast-2025/blob/main/periodic_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests xarray pandas pyjstat datetime pydap netCDF4
import os
import sqlite3
import pandas as pd
import requests
from pyjstat import pyjstat
from urllib.parse import quote
from io import StringIO
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Ensure compatibility with Colab and GitHub
!apt-get update && apt-get install -y iputils-ping

# Define a configurable database path
DB_PATH = os.getenv("DB_PATH", "greenland_fishery.db")  # Use environment variable or default to local file

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
print(f"Connected to SQLite database at {DB_PATH}")

# Function to execute SQL scripts (for DML updates)
def execute_sql_script(file_path):
    try:
        with open(file_path, 'r') as file:
            sql_script = file.read()
        cursor.executescript(sql_script)
        conn.commit()
        print(f"Successfully executed SQL script: {file_path}")
    except Exception as e:
        print(f"Error executing SQL script {file_path}: {e}")
        raise

In [ ]:
# Helper function to validate DataFrame against schema
def validate_dataframe(df, expected_columns, dtypes):
    if not all(col in df.columns for col in expected_columns):
        raise ValueError(f"DataFrame missing expected columns: {expected_columns}")
    for col, dtype in dtypes.items():
        if col in df.columns:
            df[col] = df[col].astype(dtype)
    if df.isnull().any().any():
        raise ValueError(f"DataFrame contains NaN values: {df.head()}")

In [ ]:
# UPDATE TOTAL CATCH

# Define quarter to months mapping for SST updates
quarter_to_months = {
    "Q1": ("01-01", "03-31"),
    "Q2": ("04-01", "06-30"),
    "Q3": ("07-01", "09-30"),
    "Q4": ("10-01", "12-31")
}

# Determine the most recent data point in the total_catch table
cursor.execute("SELECT MAX(Year), Quarter FROM total_catch WHERE Year = (SELECT MAX(Year) FROM total_catch)")
result = cursor.fetchone()
if result and result[0] is not None:
    latest_year, latest_quarter = result
    print(f"Latest data point: {latest_year} {latest_quarter}")
else:
    latest_year, latest_quarter = 2010, "Q4"  # Fallback to start of range
    print("No data found in total_catch, starting from 2010 Q4")

# Determine the next quarter to fetch
quarter_order = ["Q1", "Q2", "Q3", "Q4"]
next_quarter_idx = (quarter_order.index(latest_quarter) + 1) % 4
next_year = latest_year if next_quarter_idx != 0 else latest_year + 1
next_quarter = quarter_order[next_quarter_idx]
print(f"Fetching data for: {next_year} {next_quarter}")

# Fetch new data from the API
url = "https://bank.stat.gl:443/api/v1/en/Greenland/FI/FI10/FIX008.px"
query = {
    "query": [
        {"code": "nation", "selection": {"filter": "item", "values": ["GRL"]}},
        {"code": "unit", "selection": {"filter": "item", "values": ["Ton"]}},
        {"code": "time", "selection": {"filter": "item", "values": [str(next_year)]}},
        {"code": "quarter", "selection": {"filter": "item", "values": [str(next_quarter_idx + 1)]}}
    ],
    "response": {"format": "json-stat2"}
}
try:
    response = requests.post(url, json=query, timeout=30)
    response.raise_for_status()
    dataset = pyjstat.Dataset.read(response.text)
    df = dataset.write('dataframe')
    print("Data successfully retrieved and converted to DataFrame!")

    # Clean DataFrame
    df_new = df.copy()
    df_new.drop(columns=['nation'], inplace=True)
    df_new.rename(columns={
        "time": "Year",
        "quarter": "Quarter",
        "unit": "Unit",
        "value": "Total_Catch"
    }, inplace=True)
    df_new["Quarter"] = df_new["Quarter"].str.replace("Quarter ", "Q")
    df_new["Quarter"] = pd.Categorical(df_new["Quarter"], categories=quarter_order, ordered=True)
    df_new = df_new[["Year", "Quarter", "Unit", "Total_Catch"]]
    df_new["Year"] = df_new["Year"].astype(int)

    # Validate before updating SQLite
    expected_columns = ["Year", "Quarter", "Unit", "Total_Catch"]
    dtypes = {"Year": int, "Quarter": str, "Unit": str, "Total_Catch": int}
    validate_dataframe(df_new, expected_columns, dtypes)

    # Generate DML statements for insertion
    dml_statements = []
    for _, row in df_new.iterrows():
        dml_statements.append(
            f"INSERT INTO total_catch (Year, Quarter, Unit, Total_Catch) VALUES "
            f"({row['Year']}, '{row['Quarter']}', '{row['Unit']}', {row['Total_Catch']})"
        )

    # Append DML statements to dml_populate.sql
    with open('dml_populate.sql', 'a') as f:
        f.write("\n-- Update for total_catch\n")
        f.write("\n".join(dml_statements) + ";\n")

    # Execute the DML script
    execute_sql_script('dml_populate.sql')
    print("Updated total_catch table with new data")
except requests.exceptions.RequestException as e:
    print(f"Error fetching new Total Catch data: {e}")

# Final display
print("Updated Total Catch DataFrame:")
df_updated = pd.read_sql_query("SELECT * FROM total_catch WHERE Year = ? AND Quarter = ?", conn, params=(next_year, next_quarter))
display(df_updated)

In [ ]:
# UPDATE FISH EXPORTS

# Determine the most recent data point in the fish_exports table
cursor.execute("SELECT MAX(Year), Quarter FROM fish_exports WHERE Year = (SELECT MAX(Year) FROM fish_exports)")
result = cursor.fetchone()
if result and result[0] is not None:
    latest_year, latest_quarter = result
    print(f"Latest data point: {latest_year} {latest_quarter}")
else:
    latest_year, latest_quarter = 2010, "Q4"  # Fallback to start of range
    print("No data found in fish_exports, starting from 2010 Q4")

# Determine the next quarter to fetch
quarter_order = ["Q1", "Q2", "Q3", "Q4"]
next_quarter_idx = (quarter_order.index(latest_quarter) + 1) % 4
next_year = latest_year if next_quarter_idx != 0 else latest_year + 1
next_quarter = quarter_order[next_quarter_idx]
print(f"Fetching data for: {next_year} {next_quarter}")

# Fetch new data from the API
url = "https://bank.stat.gl:443/api/v1/en/Greenland/BE/BE80/BEXSTA22.px"
query = {
    "query": [
        {"code": "unit", "selection": {"filter": "item", "values": ["Mill. kr."]}},
        {"code": "time", "selection": {"filter": "item", "values": [str(next_year)]}},
        {"code": "quarter", "selection": {"filter": "item", "values": [str(next_quarter_idx + 1)]}}
    ],
    "response": {"format": "json-stat2"}
}
try:
    response = requests.post(url, json=query, timeout=30)
    response.raise_for_status()
    dataset = pyjstat.Dataset.read(response.text)
    df = dataset.write('dataframe')
    print("Data successfully retrieved and converted to DataFrame!")

    # Clean DataFrame
    df_new = df.copy()
    df_new.rename(columns={
        "time": "Year",
        "quarter": "Quarter",
        "value": "Fish_Export_Value_Million_Kr"
    }, inplace=True)
    df_new["Quarter"] = df_new["Quarter"].str.replace("Quarter ", "Q")
    df_new["Quarter"] = pd.Categorical(df_new["Quarter"], categories=quarter_order, ordered=True)
    df_new = df_new[["Year", "Quarter", "Fish_Export_Value_Million_Kr"]]
    df_new["Year"] = df_new["Year"].astype(int)

    # Validate before updating SQLite
    expected_columns = ["Year", "Quarter", "Fish_Export_Value_Million_Kr"]
    dtypes = {"Year": int, "Quarter": str, "Fish_Export_Value_Million_Kr": int}
    validate_dataframe(df_new, expected_columns, dtypes)

    # Generate DML statements for insertion
    dml_statements = []
    for _, row in df_new.iterrows():
        dml_statements.append(
            f"INSERT INTO fish_exports (Year, Quarter, Fish_Export_Value_Million_Kr) VALUES "
            f"({row['Year']}, '{row['Quarter']}', {row['Fish_Export_Value_Million_Kr']})"
        )

    # Append DML statements to dml_populate.sql
    with open('dml_populate.sql', 'a') as f:
        f.write("\n-- Update for fish_exports\n")
        f.write("\n".join(dml_statements) + ";\n")

    # Execute the DML script
    execute_sql_script('dml_populate.sql')
    print("Updated fish_exports table with new data")
except requests.exceptions.RequestException as e:
    print(f"Error fetching new Fish Exports data: {e}")

# Final display
print("Updated Fish Exports DataFrame:")
df_updated = pd.read_sql_query("SELECT * FROM fish_exports WHERE Year = ? AND Quarter = ?", conn, params=(next_year, next_quarter))
display(df_updated)

In [ ]:
# UPDATE WEST GREENLAND SST

# Degree to ERDDAP grid index conversion
def deg_to_index_lat(lat): return int(round((lat + 90) / 0.25))
def deg_to_index_lon(lon): return int(round((lon + 180) / 0.25))

# Define bounding box in degrees for West Greenland
bbox_deg_west = {
    'lat_min': 65.0,
    'lat_max': 70.0,
    'lon_min': -55.0,
    'lon_max': -50.0
}

# Convert to grid indices
bbox_idx_west = {
    'lat_min': deg_to_index_lat(bbox_deg_west['lat_min']),
    'lat_max': deg_to_index_lat(bbox_deg_west['lat_max']),
    'lon_min': deg_to_index_lon(bbox_deg_west['lon_min']),
    'lon_max': deg_to_index_lon(bbox_deg_west['lon_max'])
}

# Determine the most recent data point in the sst_west table
cursor.execute("SELECT MAX(Year), Quarter FROM sst_west WHERE Year = (SELECT MAX(Year) FROM sst_west)")
result = cursor.fetchone()
if result and result[0] is not None:
    latest_year, latest_quarter = result
    print(f"Latest data point: {latest_year} {latest_quarter}")
else:
    latest_year, latest_quarter = 2010, "Q4"  # Fallback to start of range
    print("No data found in sst_west, starting from 2010 Q4")

# Determine the next quarter to fetch
quarter_order = ["Q1", "Q2", "Q3", "Q4"]
next_quarter_idx = (quarter_order.index(latest_quarter) + 1) % 4
next_year = latest_year if next_quarter_idx != 0 else latest_year + 1
next_quarter = quarter_order[next_quarter_idx]
print(f"Fetching data for: {next_year} {next_quarter}")

# Map quarter to months
quarter_to_months = {
    "Q1": ("01-01", "03-31"),
    "Q2": ("04-01", "06-30"),
    "Q3": ("07-01", "09-30"),
    "Q4": ("10-01", "12-31")
}
start_month, end_month = quarter_to_months[next_quarter]

# Fetch new data from the API
try:
    base = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst21Agg_LonPM180.csv?"
    var = "sst"
    time = f"[({next_year}-{start_month}T00:00:00Z):1:({next_year}-{end_month}T00:00:00Z)]".replace(" ", "")
    zlev = "[0:1:0]"
    lat = f"[({bbox_idx_west['lat_min']}):1:({bbox_idx_west['lat_max']})]"
    lon = f"[({bbox_idx_west['lon_min']}):1:({bbox_idx_west['lon_max']})]"
    query = f"{var}{time}{zlev}{lat}{lon}"
    full_url = base + quote(query, safe=":/[](),-T")
    print("Constructed URL:", full_url)

    response = requests.get(full_url, timeout=30)
    response.raise_for_status()

    df = pd.read_csv(StringIO(response.text), skiprows=[1])
    df = df.rename(columns={col: col.strip() for col in df.columns})
    df = df.dropna(subset=["sst"])

    df["time"] = pd.to_datetime(df["time"])
    df["Year"] = df["time"].dt.year.astype(int)
    df["Quarter"] = "Q" + df["time"].dt.quarter.astype(str)

    df_new = df.groupby(['Year', 'Quarter'])["sst"].mean().reset_index()
    df_new = df_new.rename(columns={"sst": "Sea_Surface_Temp_C_West"})

    df_new["Melt_Active_West"] = (df_new["Sea_Surface_Temp_C_West"] > 0.5).astype(int)
    df_new["Melt_Index_West"] = df_new["Sea_Surface_Temp_C_West"].clip(lower=0, upper=4) / 4

    # Validate before updating SQLite
    expected_columns = ["Year", "Quarter", "Sea_Surface_Temp_C_West", "Melt_Active_West", "Melt_Index_West"]
    dtypes = {"Year": int, "Quarter": str, "Sea_Surface_Temp_C_West": float, "Melt_Active_West": int, "Melt_Index_West": float}
    validate_dataframe(df_new, expected_columns, dtypes)

    # Generate DML statements for insertion
    dml_statements = []
    for _, row in df_new.iterrows():
        dml_statements.append(
            f"INSERT INTO sst_west (Year, Quarter, Sea_Surface_Temp_C_West, Melt_Active_West, Melt_Index_West) VALUES "
            f"({row['Year']}, '{row['Quarter']}', {row['Sea_Surface_Temp_C_West']}, {row['Melt_Active_West']}, {row['Melt_Index_West']})"
        )

    # Append DML statements to dml_populate.sql
    with open('dml_populate.sql', 'a') as f:
        f.write("\n-- Update for sst_west\n")
        f.write("\n".join(dml_statements) + ";\n")

    # Execute the DML script
    execute_sql_script('dml_populate.sql')
    print("Updated sst_west table with new data")
except requests.exceptions.RequestException as e:
    print(f"Error fetching new SST West data: {e}")

# Final display
print("Updated SST West DataFrame:")
df_updated = pd.read_sql_query("SELECT * FROM sst_west WHERE Year = ? AND Quarter = ?", conn, params=(next_year, next_quarter))
display(df_updated)

In [ ]:
# UPDATE EAST GREENLAND SST

# Define bounding box in degrees for East Greenland
bbox_deg_east = {
    'lat_min': 65.0,
    'lat_max': 70.0,
    'lon_min': -40.0,
    'lon_max': -35.0
}

# Convert to grid indices
bbox_idx_east = {
    'lat_min': deg_to_index_lat(bbox_deg_east['lat_min']),
    'lat_max': deg_to_index_lat(bbox_deg_east['lat_max']),
    'lon_min': deg_to_index_lon(bbox_deg_east['lon_min']),
    'lon_max': deg_to_index_lon(bbox_deg_east['lon_max'])
}

# Determine the most recent data point in the sst_east table
cursor.execute("SELECT MAX(Year), Quarter FROM sst_east WHERE Year = (SELECT MAX(Year) FROM sst_east)")
result = cursor.fetchone()
if result and result[0] is not None:
    latest_year, latest_quarter = result
    print(f"Latest data point: {latest_year} {latest_quarter}")
else:
    latest_year, latest_quarter = 2010, "Q4"  # Fallback to start of range
    print("No data found in sst_east, starting from 2010 Q4")

# Determine the next quarter to fetch
quarter_order = ["Q1", "Q2", "Q3", "Q4"]
next_quarter_idx = (quarter_order.index(latest_quarter) + 1) % 4
next_year = latest_year if next_quarter_idx != 0 else latest_year + 1
next_quarter = quarter_order[next_quarter_idx]
print(f"Fetching data for: {next_year} {next_quarter}")

# Map quarter to months
start_month, end_month = quarter_to_months[next_quarter]

# Fetch new data from the API
try:
    base = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst21Agg_LonPM180.csv?"
    var = "sst"
    time = f"[({next_year}-{start_month}T00:00:00Z):1:({next_year}-{end_month}T00:00:00Z)]".replace(" ", "")
    zlev = "[0:1:0]"
    lat = f"[({bbox_idx_east['lat_min']}):1:({bbox_idx_east['lat_max']})]"
    lon = f"[({bbox_idx_east['lon_min']}):1:({bbox_idx_east['lon_max']})]"
    query = f"{var}{time}{zlev}{lat}{lon}"
    full_url = base + quote(query, safe=":/[](),-T")
    print("Constructed URL:", full_url)

    response = requests.get(full_url, timeout=30)
    response.raise_for_status()

    df = pd.read_csv(StringIO(response.text), skiprows=[1])
    df = df.rename(columns={col: col.strip() for col in df.columns})
    df = df.dropna(subset=["sst"])

    df["time"] = pd.to_datetime(df["time"])
    df["Year"] = df["time"].dt.year.astype(int)
    df["Quarter"] = "Q" + df["time"].dt.quarter.astype(str)

    df_new = df.groupby(['Year', 'Quarter'])["sst"].mean().reset_index()
    df_new = df_new.rename(columns={"sst": "Sea_Surface_Temp_C_East"})

    df_new["Melt_Active_East"] = (df_new["Sea_Surface_Temp_C_East"] > 0.5).astype(int)
    df_new["Melt_Index_East"] = df_new["Sea_Surface_Temp_C_East"].clip(lower=0, upper=4) / 4

    # Validate before updating SQLite
    expected_columns = ["Year", "Quarter", "Sea_Surface_Temp_C_East", "Melt_Active_East", "Melt_Index_East"]
    dtypes = {"Year": int, "Quarter": str, "Sea_Surface_Temp_C_East": float, "Melt_Active_East": int, "Melt_Index_East": float}
    validate_dataframe(df_new, expected_columns, dtypes)

    # Generate DML statements for insertion
    dml_statements = []
    for _, row in df_new.iterrows():
        dml_statements.append(
            f"INSERT INTO sst_east (Year, Quarter, Sea_Surface_Temp_C_East, Melt_Active_East, Melt_Index_East) VALUES "
            f"({row['Year']}, '{row['Quarter']}', {row['Sea_Surface_Temp_C_East']}, {row['Melt_Active_East']}, {row['Melt_Index_East']})"
        )

    # Append DML statements to dml_populate.sql
    with open('dml_populate.sql', 'a') as f:
        f.write("\n-- Update for sst_east\n")
        f.write("\n".join(dml_statements) + ";\n")

    # Execute the DML script
    execute_sql_script('dml_populate.sql')
    print("Updated sst_east table with new data")
except requests.exceptions.RequestException as e:
    print(f"Error fetching new SST East data: {e}")

# Final display
print("Updated SST East DataFrame:")
df_updated = pd.read_sql_query("SELECT * FROM sst_east WHERE Year = ? AND Quarter = ?", conn, params=(next_year, next_quarter))
display(df_updated)

In [ ]:
# UPDATE SOUTH GREENLAND SST

# Define bounding box in degrees for South Greenland
bbox_deg_south = {
    'lat_min': 60.0,
    'lat_max': 65.0,
    'lon_min': -45.0,
    'lon_max': -40.0
}

# Convert to grid indices
bbox_idx_south = {
    'lat_min': deg_to_index_lat(bbox_deg_south['lat_min']),
    'lat_max': deg_to_index_lat(bbox_deg_south['lat_max']),
    'lon_min': deg_to_index_lon(bbox_deg_south['lon_min']),
    'lon_max': deg_to_index_lon(bbox_deg_south['lon_max'])
}

# Determine the most recent data point in the sst_south table
cursor.execute("SELECT MAX(Year), Quarter FROM sst_south WHERE Year = (SELECT MAX(Year) FROM sst_south)")
result = cursor.fetchone()
if result and result[0] is not None:
    latest_year, latest_quarter = result
    print(f"Latest data point: {latest_year} {latest_quarter}")
else:
    latest_year, latest_quarter = 2010, "Q4"  # Fallback to start of range
    print("No data found in sst_south, starting from 2010 Q4")

# Determine the next quarter to fetch
quarter_order = ["Q1", "Q2", "Q3", "Q4"]
next_quarter_idx = (quarter_order.index(latest_quarter) + 1) % 4
next_year = latest_year if next_quarter_idx != 0 else latest_year + 1
next_quarter = quarter_order[next_quarter_idx]
print(f"Fetching data for: {next_year} {next_quarter}")

# Map quarter to months
start_month, end_month = quarter_to_months[next_quarter]

# Fetch new data from the API
try:
    base = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst21Agg_LonPM180.csv?"
    var = "sst"
    time = f"[({next_year}-{start_month}T00:00:00Z):1:({next_year}-{end_month}T00:00:00Z)]".replace(" ", "")
    zlev = "[0:1:0]"
    lat = f"[({bbox_idx_south['lat_min']}):1:({bbox_idx_south['lat_max']})]"
    lon = f"[({bbox_idx_south['lon_min']}):1:({bbox_idx_south['lon_max']})]"
    query = f"{var}{time}{zlev}{lat}{lon}"
    full_url = base + quote(query, safe=":/[](),-T")
    print("Constructed URL:", full_url)

    response = requests.get(full_url, timeout=30)
    response.raise_for_status()

    df = pd.read_csv(StringIO(response.text), skiprows=[1])
    df = df.rename(columns={col: col.strip() for col in df.columns})
    df = df.dropna(subset=["sst"])

    df["time"] = pd.to_datetime(df["time"])
    df["Year"] = df["time"].dt.year.astype(int)
    df["Quarter"] = "Q" + df["time"].dt.quarter.astype(str)

    df_new = df.groupby(['Year', 'Quarter'])["sst"].mean().reset_index()
    df_new = df_new.rename(columns={"sst": "Sea_Surface_Temp_C_South"})

    df_new["Melt_Active_South"] = (df_new["Sea_Surface_Temp_C_South"] > 0.5).astype(int)
    df_new["Melt_Index_South"] = df_new["Sea_Surface_Temp_C_South"].clip(lower=0, upper=4) / 4

    # Validate before updating SQLite
    expected_columns = ["Year", "Quarter", "Sea_Surface_Temp_C_South", "Melt_Active_South", "Melt_Index_South"]
    dtypes = {"Year": int, "Quarter": str, "Sea_Surface_Temp_C_South": float, "Melt_Active_South": int, "Melt_Index_South": float}
    validate_dataframe(df_new, expected_columns, dtypes)

    # Generate DML statements for insertion
    dml_statements = []
    for _, row in df_new.iterrows():
        dml_statements.append(
            f"INSERT INTO sst_south (Year, Quarter, Sea_Surface_Temp_C_South, Melt_Active_South, Melt_Index_South) VALUES "
            f"({row['Year']}, '{row['Quarter']}', {row['Sea_Surface_Temp_C_South']}, {row['Melt_Active_South']}, {row['Melt_Index_South']})"
        )

    # Append DML statements to dml_populate.sql
    with open('dml_populate.sql', 'a') as f:
        f.write("\n-- Update for sst_south\n")
        f.write("\n".join(dml_statements) + ";\n")

    # Execute the DML script
    execute_sql_script('dml_populate.sql')
    print("Updated sst_south table with new data")
except requests.exceptions.RequestException as e:
    print(f"Error fetching new SST South data: {e}")

# Final display
print("Updated SST South DataFrame:")
df_updated = pd.read_sql_query("SELECT * FROM sst_south WHERE Year = ? AND Quarter = ?", conn, params=(next_year, next_quarter))
display(df_updated)

In [ ]:
# UPDATE FOREIGN CATCH

# Determine the most recent data point in the foreign_catch table
cursor.execute("SELECT MAX(Year), Quarter FROM foreign_catch WHERE Year = (SELECT MAX(Year) FROM foreign_catch)")
result = cursor.fetchone()
if result and result[0] is not None:
    latest_year, latest_quarter = result
    print(f"Latest data point: {latest_year} {latest_quarter}")
else:
    latest_year, latest_quarter = 2010, "Q4"  # Fallback to start of range
    print("No data found in foreign_catch, starting from 2010 Q4")

# Determine the next quarter to fetch
quarter_order = ["Q1", "Q2", "Q3", "Q4"]
next_quarter_idx = (quarter_order.index(latest_quarter) + 1) % 4
next_year = latest_year if next_quarter_idx != 0 else latest_year + 1
next_quarter = quarter_order[next_quarter_idx]
print(f"Fetching data for: {next_year} {next_quarter}")

# Fetch new data from the API
url = "https://bank.stat.gl:443/api/v1/en/Greenland/FI/FI10/FIX008.px"
query = {
    "query": [
        {"code": "nation", "selection": {"filter": "item", "values": ["Foreign"]}},
        {"code": "unit", "selection": {"filter": "item", "values": ["Ton"]}},
        {"code": "time", "selection": {"filter": "item", "values": [str(next_year)]}},
        {"code": "quarter", "selection": {"filter": "item", "values": [str(next_quarter_idx + 1)]}}
    ],
    "response": {"format": "json-stat2"}
}
try:
    response = requests.post(url, json=query, timeout=30)
    response.raise_for_status()
    dataset = pyjstat.Dataset.read(response.text)
    df = dataset.write('dataframe')
    print("Data successfully retrieved and converted to DataFrame!")

    # Clean DataFrame
    df_new = df.copy()
    df_new.drop(columns=['nation'], inplace=True)
    df_new.rename(columns={
        "time": "Year",
        "quarter": "Quarter",
        "unit": "Unit",
        "value": "Foreign_Catch"
    }, inplace=True)
    df_new["Quarter"] = df_new["Quarter"].str.replace("Quarter ", "Q")
    df_new["Quarter"] = pd.Categorical(df_new["Quarter"], categories=quarter_order, ordered=True)
    df_new = df_new[["Year", "Quarter", "Unit", "Foreign_Catch"]]
    df_new["Year"] = df_new["Year"].astype(int)

    # Validate before updating SQLite
    expected_columns = ["Year", "Quarter", "Unit", "Foreign_Catch"]
    dtypes = {"Year": int, "Quarter": str, "Unit": str, "Foreign_Catch": int}
    validate_dataframe(df_new, expected_columns, dtypes)

    # Generate DML statements for insertion
    dml_statements = []
    for _, row in df_new.iterrows():
        dml_statements.append(
            f"INSERT INTO foreign_catch (Year, Quarter, Unit, Foreign_Catch) VALUES "
            f"({row['Year']}, '{row['Quarter']}', '{row['Unit']}', {row['Foreign_Catch']})"
        )

    # Append DML statements to dml_populate.sql
    with open('dml_populate.sql', 'a') as f:
        f.write("\n-- Update for foreign_catch\n")
        f.write("\n".join(dml_statements) + ";\n")

    # Execute the DML script
    execute_sql_script('dml_populate.sql')
    print("Updated foreign_catch table with new data")
except requests.exceptions.RequestException as e:
    print(f"Error fetching new Foreign Catch data: {e}")

# Final display
print("Updated Foreign Catch DataFrame:")
df_updated = pd.read_sql_query("SELECT * FROM foreign_catch WHERE Year = ? AND Quarter = ?", conn, params=(next_year, next_quarter))
display(df_updated)

In [ ]:
# UPDATE ICE MELT SST

# Determine the most recent data point in the ice_melt_sst table
cursor.execute("SELECT MAX(Year), Quarter FROM ice_melt_sst WHERE Year = (SELECT MAX(Year) FROM ice_melt_sst)")
result = cursor.fetchone()
if result and result[0] is not None:
    latest_year, latest_quarter = result
    print(f"Latest data point: {latest_year} {latest_quarter}")
else:
    latest_year, latest_quarter = 2010, "Q4"  # Fallback to start of range
    print("No data found in ice_melt_sst, starting from 2010 Q4")

# Determine the next quarter to fetch
quarter_order = ["Q1", "Q2", "Q3", "Q4"]
next_quarter_idx = (quarter_order.index(latest_quarter) + 1) % 4
next_year = latest_year if next_quarter_idx != 0 else latest_year + 1
next_quarter = quarter_order[next_quarter_idx]
print(f"Computing data for: {next_year} {next_quarter}")

# Fetch the latest SST data for East and West Greenland
df_sst_east = pd.read_sql_query("SELECT * FROM sst_east WHERE Year = ? AND Quarter = ?", conn, params=(next_year, next_quarter))
df_sst_west = pd.read_sql_query("SELECT * FROM sst_west WHERE Year = ? AND Quarter = ?", conn, params=(next_year, next_quarter))

if not df_sst_east.empty and not df_sst_west.empty:
    # Normalize SST for the formula (assuming SST ranges from -2 to 20°C, map to 0–1)
    sst_east_norm = (df_sst_east["Sea_Surface_Temp_C_East"].iloc[0] + 2) / 22
    sst_west_norm = (df_sst_west["Sea_Surface_Temp_C_West"].iloc[0] + 2) / 22

    # Compute Ice Melt Rate as a weighted average of Melt Index and normalized SST
    ice_melt_rate_east = (0.7 * df_sst_east["Melt_Index_East"].iloc[0] + 0.3 * sst_east_norm)
    ice_melt_rate_west = (0.7 * df_sst_west["Melt_Index_West"].iloc[0] + 0.3 * sst_west_norm)

    # Create DataFrame
    df_new = pd.DataFrame({
        "Year": [next_year],
        "Quarter": [next_quarter],
        "Ice_Melt_Rate_East": [ice_melt_rate_east],
        "Ice_Melt_Rate_West": [ice_melt_rate_west],
        "SST_East": [df_sst_east["Sea_Surface_Temp_C_East"].iloc[0]],
        "SST_West": [df_sst_west["Sea_Surface_Temp_C_West"].iloc[0]]
    })

    # Validate before updating SQLite
    expected_columns = ["Year", "Quarter", "Ice_Melt_Rate_East", "Ice_Melt_Rate_West", "SST_East", "SST_West"]
    dtypes = {"Year": int, "Quarter": str, "Ice_Melt_Rate_East": float, "Ice_Melt_Rate_West": float, "SST_East": float, "SST_West": float}
    validate_dataframe(df_new, expected_columns, dtypes)

    # Generate DML statements for insertion
    dml_statements = []
    for _, row in df_new.iterrows():
        dml_statements.append(
            f"INSERT INTO ice_melt_sst (Year, Quarter, Ice_Melt_Rate_East, Ice_Melt_Rate_West, SST_East, SST_West) VALUES "
            f"({row['Year']}, '{row['Quarter']}', {row['Ice_Melt_Rate_East']}, {row['Ice_Melt_Rate_West']}, {row['SST_East']}, {row['SST_West']})"
        )

    # Append DML statements to dml_populate.sql
    with open('dml_populate.sql', 'a') as f:
        f.write("\n-- Update for ice_melt_sst\n")
        f.write("\n".join(dml_statements) + ";\n")

    # Execute the DML script
    execute_sql_script('dml_populate.sql')
    print("Updated ice_melt_sst table with new data")
else:
    print("No new data available for ice_melt_sst (requires updated sst_east and sst_west data)")

# Final display
print("Updated Ice Melt SST DataFrame:")
df_updated = pd.read_sql_query("SELECT * FROM ice_melt_sst WHERE Year = ? AND Quarter = ?", conn, params=(next_year, next_quarter))
display(df_updated)

In [ ]:
# Close the database connection
conn.close()
print("Database connection closed.")